<a href="https://colab.research.google.com/github/mhyls26/CCMACLRL_EXERCISES_COM231ML/blob/main/Exercise%206%20/%20Exercise6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 6: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all Regression models

Submit your results to:
https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview



In [140]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

## Dataset File

In [141]:
train_data = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/train.csv?raw=true'
df = pd.read_csv(train_data)

## Test File

In [142]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [143]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

## Sample Submission File

In [144]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/3fd7d51ffd17863598ac3f44eeefc558171a5b73/dataset/house-prices-advanced-regression-techniques/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [145]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Id         1459 non-null   int64  
 1   SalePrice  1459 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 22.9 KB


In [146]:
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna("Missing")
    dt[col] = dt[col].fillna("Missing")

    le = LabelEncoder()
    le.fit(pd.concat([df[col], dt[col]], axis=0))

    df[col] = le.transform(df[col])
    dt[col] = le.transform(dt[col])

In [147]:
for col in df.select_dtypes(include=[np.number]).columns:
    df[col] = df[col].fillna(df[col].median())
for col in dt.select_dtypes(include=[np.number]).columns:
    dt[col] = dt[col].fillna(dt[col].median())

In [148]:
X = df.drop(["SalePrice", "Id"], axis=1)
y = df["SalePrice"]

In [149]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## 1. Train a KNN Regressor

In [150]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
knn_rmse = np.sqrt(mean_squared_error(y_val, knn_pred))
print("KNN RMSE:", knn_rmse)

KNN RMSE: 47678.1071324634


- Perform cross validation

In [151]:
knn_cv_rmse = -cross_val_score(knn, X, y, cv=5, scoring="neg_root_mean_squared_error").mean()
print("KNN CV RMSE:", knn_cv_rmse)

KNN CV RMSE: 47004.09670873701


## 2. Train a SVM Regression

In [152]:
svm = SVR(kernel="rbf")
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
svm_rmse = np.sqrt(mean_squared_error(y_val, svm_pred))
print("SVM RMSE:", svm_rmse)

SVM RMSE: 88652.804331602


- Perform cross validation

In [153]:
svm_cv_rmse = -cross_val_score(svm, X, y, cv=5, scoring="neg_root_mean_squared_error").mean()
print("SVM CV RMSE:", svm_cv_rmse)

SVM CV RMSE: 81143.80317974449


## 3. Train a Decision Tree Regression

In [154]:
dtree = DecisionTreeRegressor(random_state=42)
dtree.fit(X_train, y_train)
dtree_pred = dtree.predict(X_val)
dtree_rmse = np.sqrt(mean_squared_error(y_val, dtree_pred))
print("Decision Tree RMSE:", dtree_rmse)

Decision Tree RMSE: 44508.93235921133


- Perform cross validation

In [155]:
dtree_cv_rmse = -cross_val_score(dtree, X, y, cv=5, scoring="neg_root_mean_squared_error").mean()
print("Decision Tree CV RMSE:", dtree_cv_rmse)

Decision Tree CV RMSE: 42703.92723506582


## 4. Train a Random Forest Regression

In [156]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
print("Random Forest RMSE:", rf_rmse)

Random Forest RMSE: 28470.690412167223


- Perform cross validation

In [157]:
rf_cv_rmse = -cross_val_score(rf, X, y, cv=5, scoring="neg_root_mean_squared_error").mean()
print("Random Forest CV RMSE:", rf_cv_rmse)

Random Forest CV RMSE: 29740.489630520016


## 5. Compare all the performance of all regression models

In [158]:
results = pd.DataFrame({
    "Model": ["KNN", "SVM", "Decision Tree", "Random Forest"],
    "Validation RMSE": [knn_rmse, svm_rmse, dtree_rmse, rf_rmse],
    "CrossVal RMSE": [knn_cv_rmse, svm_cv_rmse, dtree_cv_rmse, rf_cv_rmse]
})

print("\n--- Model Comparison ---")
print(results)


--- Model Comparison ---
           Model  Validation RMSE  CrossVal RMSE
0            KNN     47678.107132   47004.096709
1            SVM     88652.804332   81143.803180
2  Decision Tree     44508.932359   42703.927235
3  Random Forest     28470.690412   29740.489631


## 6. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [159]:
submission_df = pd.DataFrame({
    "Id": id_col,
    "SalePrice": y_pred
})

submission_df.to_csv("submission.csv", index=False)
print("Submission file created in correct format: submission.csv")

Submission file created in correct format: submission.csv
